In [7]:
pip install chardet

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import chardet
import re


def detect_encoding(file_path):
    """
    检测文件编码
    """
    with open(file_path, 'rb') as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        confidence = result['confidence']
        print(f"检测到文件编码: {encoding} (置信度: {confidence})")
        return encoding
    
def get_administrative_divisions(url):
    """
    从民政部网站获取行政区划数据
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        print("正在从民政部网站获取行政区划数据...")
        response = requests.get(url, headers=headers, timeout=10)
        response.encoding = 'utf-8'
        
        if response.status_code != 200:
            print(f"请求失败，状态码: {response.status_code}")
            return {}
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 查找包含行政区划数据的表格
        tables = soup.find_all('table')
        administrative_data = {}
        
        for table in tables:
            rows = table.find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                if len(cells) >= 2:
                    # 提取代码和名称
                    code = cells[0].get_text().strip()
                    name = cells[1].get_text().strip()
                    
                    # 省级行政区划代码以0000结尾
                    if code.endswith('0000') and name:
                        administrative_data[name] = name
                        
                        # 特别处理直辖市和特别行政区
                        if name in ['北京市', '天津市', '上海市', '重庆市']:
                            administrative_data[name[:-1]] = name
                        elif name in ['香港特别行政区', '澳门特别行政区']:
                            administrative_data[name.replace('特别行政区', '')] = name
                            
        print(f"成功获取 {len(administrative_data)} 个行政区划数据")
        return administrative_data
        
    except Exception as e:
        print(f"获取行政区划数据时出错: {e}")
        print("将使用内置的行政区划数据...")
        # 如果网络请求失败，使用内置的行政区划数据
        return get_fallback_administrative_data()
    
def get_city_province_mapping():
    """
    创建城市到省份的映射关系
    """
    # 完整的城市-省份映射
    city_province_map = {
        # 直辖市
        "北京": "北京市", "上海": "上海市", "天津": "天津市", "重庆": "重庆市",
        
        # 省份及省会/主要城市
        "石家庄": "河北省", "唐山": "河北省", "秦皇岛": "河北省", "邯郸": "河北省", "邢台": "河北省", 
        "保定": "河北省", "张家口": "河北省", "承德": "河北省", "沧州": "河北省", "廊坊": "河北省", 
        "衡水": "河北省", "太原": "山西省", "大同": "山西省", "阳泉": "山西省", "长治": "山西省", 
        "晋城": "山西省", "朔州": "山西省", "晋中": "山西省", "运城": "山西省", "忻州": "山西省", 
        "临汾": "山西省", "吕梁": "山西省", "呼和浩特": "内蒙古自治区", "包头": "内蒙古自治区", 
        "乌海": "内蒙古自治区", "赤峰": "内蒙古自治区", "通辽": "内蒙古自治区", "鄂尔多斯": "内蒙古自治区", 
        "呼伦贝尔": "内蒙古自治区", "巴彦淖尔": "内蒙古自治区", "乌兰察布": "内蒙古自治区", 
        "沈阳": "辽宁省", "大连": "辽宁省", "鞍山": "辽宁省", "抚顺": "辽宁省", "本溪": "辽宁省", 
        "丹东": "辽宁省", "锦州": "辽宁省", "营口": "辽宁省", "阜新": "辽宁省", "辽阳": "辽宁省", 
        "盘锦": "辽宁省", "铁岭": "辽宁省", "朝阳": "辽宁省", "葫芦岛": "辽宁省", "长春": "吉林省", 
        "吉林": "吉林省", "四平": "吉林省", "辽源": "吉林省", "通化": "吉林省", "白山": "吉林省", 
        "松原": "吉林省", "白城": "吉林省", "延边": "吉林省", "哈尔滨": "黑龙江省", "齐齐哈尔": "黑龙江省", 
        "鸡西": "黑龙江省", "鹤岗": "黑龙江省", "双鸭山": "黑龙江省", "大庆": "黑龙江省", "伊春": "黑龙江省", 
        "佳木斯": "黑龙江省", "七台河": "黑龙江省", "牡丹江": "黑龙江省", "黑河": "黑龙江省", 
        "绥化": "黑龙江省", "大兴安岭": "黑龙江省", "南京": "江苏省", "无锡": "江苏省", "徐州": "江苏省", 
        "常州": "江苏省", "苏州": "江苏省", "南通": "江苏省", "连云港": "江苏省", "淮安": "江苏省", 
        "盐城": "江苏省", "扬州": "江苏省", "镇江": "江苏省", "泰州": "江苏省", "宿迁": "江苏省", 
        "杭州": "浙江省", "宁波": "浙江省", "温州": "浙江省", "嘉兴": "浙江省", "湖州": "浙江省", 
        "绍兴": "浙江省", "金华": "浙江省", "衢州": "浙江省", "舟山": "浙江省", "台州": "浙江省", 
        "丽水": "浙江省", "合肥": "安徽省", "芜湖": "安徽省", "蚌埠": "安徽省", "淮南": "安徽省", 
        "马鞍山": "安徽省", "淮北": "安徽省", "铜陵": "安徽省", "安庆": "安徽省", "黄山": "安徽省", 
        "滁州": "安徽省", "阜阳": "安徽省", "宿州": "安徽省", "六安": "安徽省", "亳州": "安徽省", 
        "池州": "安徽省", "宣城": "安徽省", "福州": "福建省", "厦门": "福建省", "莆田": "福建省", 
        "三明": "福建省", "泉州": "福建省", "漳州": "福建省", "南平": "福建省", "龙岩": "福建省", 
        "宁德": "福建省", "南昌": "江西省", "景德镇": "江西省", "萍乡": "江西省", "九江": "江西省", 
        "新余": "江西省", "鹰潭": "江西省", "赣州": "江西省", "吉安": "江西省", "宜春": "江西省", 
        "抚州": "江西省", "上饶": "江西省", "济南": "山东省", "青岛": "山东省", "淄博": "山东省", 
        "枣庄": "山东省", "东营": "山东省", "烟台": "山东省", "潍坊": "山东省", "济宁": "山东省", 
        "泰安": "山东省", "威海": "山东省", "日照": "山东省", "临沂": "山东省", "德州": "山东省", 
        "聊城": "山东省", "滨州": "山东省", "菏泽": "山东省", "郑州": "河南省", "开封": "河南省", 
        "洛阳": "河南省", "平顶山": "河南省", "安阳": "河南省", "鹤壁": "河南省", "新乡": "河南省", 
        "焦作": "河南省", "濮阳": "河南省", "许昌": "河南省", "漯河": "河南省", "三门峡": "河南省", 
        "南阳": "河南省", "商丘": "河南省", "信阳": "河南省", "周口": "河南省", "驻马店": "河南省", 
        "武汉": "湖北省", "黄石": "湖北省", "十堰": "湖北省", "宜昌": "湖北省", "襄阳": "湖北省", 
        "鄂州": "湖北省", "荆门": "湖北省", "孝感": "湖北省", "荆州": "湖北省", "黄冈": "湖北省", 
        "咸宁": "湖北省", "随州": "湖北省", "恩施": "湖北省", "长沙": "湖南省", "株洲": "湖南省", 
        "湘潭": "湖南省", "衡阳": "湖南省", "邵阳": "湖南省", "岳阳": "湖南省", "常德": "湖南省", 
        "张家界": "湖南省", "益阳": "湖南省", "郴州": "湖南省", "永州": "湖南省", "怀化": "湖南省", 
        "娄底": "湖南省", "湘西": "湖南省", "广州": "广东省", "韶关": "广东省", "深圳": "广东省", 
        "珠海": "广东省", "汕头": "广东省", "佛山": "广东省", "江门": "广东省", "湛江": "广东省", 
        "茂名": "广东省", "肇庆": "广东省", "惠州": "广东省", "梅州": "广东省", "汕尾": "广东省", 
        "河源": "广东省", "阳江": "广东省", "清远": "广东省", "东莞": "广东省", "中山": "广东省", 
        "潮州": "广东省", "揭阳": "广东省", "云浮": "广东省", "南宁": "广西壮族自治区", "柳州": "广西壮族自治区", 
        "桂林": "广西壮族自治区", "梧州": "广西壮族自治区", "北海": "广西壮族自治区", "防城港": "广西壮族自治区", 
        "钦州": "广西壮族自治区", "贵港": "广西壮族自治区", "玉林": "广西壮族自治区", "百色": "广西壮族自治区", 
        "贺州": "广西壮族自治区", "河池": "广西壮族自治区", "来宾": "广西壮族自治区", "崇左": "广西壮族自治区", 
        "海口": "海南省", "三亚": "海南省", "三沙": "海南省", "儋州": "海南省", "五指山": "海南省", 
        "琼海": "海南省", "文昌": "海南省", "万宁": "海南省", "东方": "海南省", "定安": "海南省", 
        "屯昌": "海南省", "澄迈": "海南省", "临高": "海南省", "白沙": "海南省", "昌江": "海南省", 
        "乐东": "海南省", "陵水": "海南省", "保亭": "海南省", "琼中": "海南省", "成都": "四川省", 
        "自贡": "四川省", "攀枝花": "四川省", "泸州": "四川省", "德阳": "四川省", "绵阳": "四川省", 
        "广元": "四川省", "遂宁": "四川省", "内江": "四川省", "乐山": "四川省", "南充": "四川省", 
        "眉山": "四川省", "宜宾": "四川省", "广安": "四川省", "达州": "四川省", "雅安": "四川省", 
        "巴中": "四川省", "资阳": "四川省", "阿坝": "四川省", "甘孜": "四川省", "凉山": "四川省", 
        "贵阳": "贵州省", "六盘水": "贵州省", "遵义": "贵州省", "安顺": "贵州省", "毕节": "贵州省", 
        "铜仁": "贵州省", "黔西南": "贵州省", "黔东南": "贵州省", "黔南": "贵州省", "昆明": "云南省", 
        "曲靖": "云南省", "玉溪": "云南省", "保山": "云南省", "昭通": "云南省", "丽江": "云南省", 
        "普洱": "云南省", "临沧": "云南省", "楚雄": "云南省", "红河": "云南省", "文山": "云南省", 
        "西双版纳": "云南省", "大理": "云南省", "德宏": "云南省", "怒江": "云南省", "迪庆": "云南省", 
        "拉萨": "西藏自治区", "昌都": "西藏自治区", "山南": "西藏自治区", "日喀则": "西藏自治区", 
        "那曲": "西藏自治区", "阿里": "西藏自治区", "林芝": "西藏自治区", "西安": "陕西省", 
        "铜川": "陕西省", "宝鸡": "陕西省", "咸阳": "陕西省", "渭南": "陕西省", "延安": "陕西省", 
        "汉中": "陕西省", "榆林": "陕西省", "安康": "陕西省", "商洛": "陕西省", "兰州": "甘肃省", 
        "嘉峪关": "甘肃省", "金昌": "甘肃省", "白银": "甘肃省", "天水": "甘肃省", "武威": "甘肃省", 
        "张掖": "甘肃省", "平凉": "甘肃省", "酒泉": "甘肃省", "庆阳": "甘肃省", "定西": "甘肃省", 
        "陇南": "甘肃省", "临夏": "甘肃省", "甘南": "甘肃省", "西宁": "青海省", "海东": "青海省", 
        "海北": "青海省", "黄南": "青海省", "海南": "青海省", "果洛": "青海省", "玉树": "青海省", 
        "海西": "青海省", "银川": "宁夏回族自治区", "石嘴山": "宁夏回族自治区", "吴忠": "宁夏回族自治区", 
        "固原": "宁夏回族自治区", "中卫": "宁夏回族自治区", "乌鲁木齐": "新疆维吾尔自治区", 
        "克拉玛依": "新疆维吾尔自治区", "吐鲁番": "新疆维吾尔自治区", "哈密": "新疆维吾尔自治区", 
        "昌吉": "新疆维吾尔自治区", "博尔塔拉": "新疆维吾尔自治区", "巴音郭楞": "新疆维吾尔自治区", 
        "阿克苏": "新疆维吾尔自治区", "克孜勒苏": "新疆维吾尔自治区", "喀什": "新疆维吾尔自治区", 
        "和田": "新疆维吾尔自治区", "伊犁": "新疆维吾尔自治区", "塔城": "新疆维吾尔自治区", 
        "阿勒泰": "新疆维吾尔自治区", "石河子": "新疆维吾尔自治区", "阿拉尔": "新疆维吾尔自治区", 
        "图木舒克": "新疆维吾尔自治区", "五家渠": "新疆维吾尔自治区", "北屯": "新疆维吾尔自治区", 
        "铁门关": "新疆维吾尔自治区", "双河": "新疆维吾尔自治区", "可克达拉": "新疆维吾尔自治区", 
        "昆玉": "新疆维吾尔自治区", "胡杨河": "新疆维吾尔自治区", "新星": "新疆维吾尔自治区",
        
        # 特别行政区
        "香港": "香港特别行政区", "澳门": "澳门特别行政区",
        
        # 其他常见旅游城市
        "都江堰": "四川省", "峨眉山": "四川省", "乐山": "四川省", "九寨沟": "四川省", 
        "黄龙": "四川省", "青城山": "四川省", "剑阁": "四川省", "阆中": "四川省", 
        "稻城": "四川省", "亚丁": "四川省", "四姑娘山": "四川省", "海螺沟": "四川省", 
        "西昌": "四川省", "泸沽湖": "四川省", "蜀南竹海": "四川省", "兴文": "四川省", 
        "华蓥": "四川省", "广汉": "四川省", "江油": "四川省", "邛崃": "四川省", 
        "崇州": "四川省", "彭州": "四川省", "简阳": "四川省", "什邡": "四川省", 
        "绵竹": "四川省", "江津": "重庆市", "合川": "重庆市", "永川": "重庆市", 
        "南川": "重庆市", "綦江": "重庆市", "大足": "重庆市", "璧山": "重庆市", 
        "铜梁": "重庆市", "潼南": "重庆市", "荣昌": "重庆市", "开州": "重庆市", 
        "梁平": "重庆市", "武隆": "重庆市", "城口": "重庆市", "丰都": "重庆市", 
        "垫江": "重庆市", "忠县": "重庆市", "云阳": "重庆市", "奉节": "重庆市", 
        "巫山": "重庆市", "巫溪": "重庆市", "石柱": "重庆市", "秀山": "重庆市", 
        "酉阳": "重庆市", "彭水": "重庆市", "万州": "重庆市", "涪陵": "重庆市", 
        "黔江": "重庆市", "长寿": "重庆市", "巴南": "重庆市", "北碚": "重庆市", 
        "璧山": "重庆市", "大渡口": "重庆市", "江北": "重庆市", "九龙坡": "重庆市", 
        "南岸": "重庆市", "沙坪坝": "重庆市", "渝中": "重庆市", "渝北": "重庆市", 
        "万盛": "重庆市", "双桥": "重庆市", "巴南": "重庆市", "武隆": "重庆市", 
        "丰都": "重庆市", "奉节": "重庆市", "云阳": "重庆市", "巫山": "重庆市", 
        "巫溪": "重庆市", "石柱": "重庆市", "秀山": "重庆市", "酉阳": "重庆市", 
        "彭水": "重庆市", "黔江": "重庆市", "綦江": "重庆市", "潼南": "重庆市", 
        "铜梁": "重庆市", "大足": "重庆市", "荣昌": "重庆市", "璧山": "重庆市", 
        "梁平": "重庆市", "城口": "重庆市", "丰都": "重庆市", "垫江": "重庆市", 
        "武隆": "重庆市", "忠县": "重庆市", "开州": "重庆市", "云阳": "重庆市", 
        "奉节": "重庆市", "巫山": "重庆市", "巫溪": "重庆市", "黔江": "重庆市", 
        "石柱": "重庆市", "秀山": "重庆市", "酉阳": "重庆市", "彭水": "重庆市",
        
        # 新添加的城市
        "澄江": "云南省", "贵德": "青海省", "尚志": "黑龙江省", "抚松": "吉林省", "莎车": "新疆维吾尔自治区",
        "兴化": "江苏省", "弥勒": "云南省", "安图": "吉林省", "宝兴": "四川省", "天全": "四川省",
        "芒市": "云南省", "库车": "新疆维吾尔自治区", "建水": "云南省", "佛坪": "陕西省", "商南": "陕西省",
        "围场": "河北省", "平邑": "山东省", "留坝": "陕西省", "阿克陶": "新疆维吾尔自治区", "秭归": "湖北省",
        "罗源": "福建省", "勉县": "陕西省", "茂县": "四川省", "峨边": "四川省", "盐源": "四川省",
        "绩溪": "安徽省", "瑞金": "江西省", "洪江": "湖南省", "曲沃": "山西省", "修文": "贵州省","饶平": "广东省",
        "泾县": "安徽省", "景洪": "云南省", "惠东": "广东省", "岱山": "浙江省", "涞源": "河北省","五台": "山西省",
        "江口": "贵州省", "芮城": "山西省", "玉龙": "云南省", "武夷山": "福建省", "揭西": "广东省","比如": "西藏自治区",
        "镇宁": "贵州省", "香格里拉": "云南省", "察右后旗": "内蒙古自治区", "广南": "云南省","达茂旗": "内蒙古自治区",
        "瓜州": "甘肃省", "北川": "四川省", "华阴": "陕西省", "若羌": "新疆维吾尔自治区","江孜": "西藏自治区", "涞水": "河北省",
        "台北": "台湾省", "庐山": "江西省", "康定": "四川省", "凤城": "辽宁省", "绥阳": "贵州省","山丹": "甘肃省",
        "阿尔山": "内蒙古自治区", "延吉": "吉林省", "瑞丽": "云南省", "泗洪": "江苏省","澜沧": "云南省", "罗成": "广西壮族自治区",
        "韶山": "湖南省", "义县": "辽宁省", "巍山": "云南省", "炎陵": "湖南省", "禄劝": "云南省","陆良": "云南省",
        "抚远": "黑龙江省", "额尔古纳": "内蒙古自治区", "勐海": "云南省", "赤水": "贵州省","东台": "江苏省", "龙南": "江西省",
        "安吉": "浙江省", "资兴": "湖南省", "凤阳": "安徽省", "巴楚": "新疆维吾尔自治区", "句容": "江苏省","永泰": "福建省",
        "神农架": "湖北省", "淳安": "浙江省", "高雄": "台湾省", "黟县": "安徽省", "太仓": "江苏省","多伦": "内蒙古自治区",
        "格尔木": "青海省", "尼勒克": "新疆维吾尔自治区", "青州": "山东省", "兴平": "陕西省","永城": "河南省",
        "富蕴": "新疆维吾尔自治区", "泰宁": "福建省", "乌兰浩特": "内蒙古自治区", "万荣": "山西省","陵川": "山西省",
        "神木": "陕西省", "晋江": "福建省", "浮梁": "江西省", "肃南": "甘肃省", "紫云": "贵州省","吴桥": "河北省",
        "克什克腾旗": "内蒙古自治区", "东兴": "广西壮族自治区", "元谋": "云南省", "海宁": "浙江省","容城": "河北省",
        "兴山": "湖北省", "昭苏": "新疆维吾尔自治区", "锡林浩特": "内蒙古自治区", "新疆": "新疆维吾尔自治区","舒城": "安徽省",
        "敦煌": "甘肃省", "石狮": "福建省", "伊吾": "新疆维吾尔自治区", "大邑": "四川省","远安": "湖北省", "颍上": "安徽省",
        "尉犁": "新疆维吾尔自治区", "昆山": "江苏省", "芦溪": "江西省", "龙井": "吉林省", "宁武": "山西省","固阳": "内蒙古自治区",
        "慈溪": "浙江省", "利川": "湖北省", "巴马": "广西壮族自治区", "登封": "河南省", "巩义": "河南省","桃园": "台湾省",
        "湖口": "江西省", "建始": "湖北省", "井陉": "河北省","霍城": "新疆维吾尔自治区","禄丰": "云南省","象山": "广西壮族自治区",
        "青川": "四川省","睢宁": "江苏省","泽普": "新疆维吾尔自治区","祁连": "甘肃省","汉寿": "湖南省","沂南": "山东省",
        "和龙": "吉林省","宣威": "云南省","米易": "四川省","武义": "浙江省","长汀": "福建省","霍尔果斯": "新疆维吾尔自治区",
        "土默特右旗": "内蒙古自治区","科左中旗": "内蒙古自治区","安福": "江西省","微山": "山东省","仙游": "福建省",
        "赵县": "河北省","缙云": "浙江省","徐闻": "广东省","汾阳": "山西省","隆昌": "四川省","嵩县": "河南省",
        "色达": "四川省","龙口": "山东省","开远": "云南省","蓝山": "云南省","仁怀": "贵州省","凯里": "贵州省",
        "库伦旗": "内蒙古自治区","河津": "山西省","施秉": "贵州省","诸暨": "浙江省","古丈": "湖南省","集安": "吉林省",
        "延川": "陕西省","定结": "西藏自治区","永靖": "甘肃省","和龙": "吉林省","金塔": "云南省","遂平": "河南省",

        
        # 更多城市
        "博乐": "新疆维吾尔自治区", "布尔津": "新疆维吾尔自治区", "阜康": "新疆维吾尔自治区", "绥芬河": "黑龙江省",
        "吉县": "山西省", "栾川": "河南省", "浑源": "山西省", "小金": "四川省", "连城": "福建省",
        "应县": "山西省", "松潘": "四川省", "阜康": "新疆维吾尔自治区", "中牟": "河南省","南投": "台湾省", 
        "祁县": "山西省", "海林": "黑龙江省", "灵石": "山西省", "壶关": "山西省", "宜兴": "江苏省",
        "雷山": "贵州省", "勐腊": "云南省", "宁蒗": "云南省", "修武": "河南省", "开化": "浙江省",
        "松潘": "四川省", "小金": "四川省", "青阳": "安徽省", "织金": "贵州省", "刚察": "青海省",
        "中牟": "河南省", "仙居": "浙江省", "栾川": "河南省", "修武": "河南省", "宁武": "山西省",
        "吉县": "山西省", "浑源": "山西省", "祁县": "山西省", "海林": "黑龙江省", "浏阳": "湖南省",
        "灵石": "山西省", "壶关": "山西省", "雷山": "贵州省", "勐腊": "云南省", "洱源": "云南省",
        "宁蒗": "云南省", "松潘": "四川省", "小金": "四川省", "青阳": "安徽省", "井冈山": "江西省",
        "织金": "贵州省", "中牟": "河南省", "仙居": "浙江省", "栾川": "河南省", "遵化": "河北省",
        "修武": "河南省", "吉县": "山西省", "浑源": "山西省", "祁县": "山西省", "凤凰": "湖南省",
        "海林": "黑龙江省", "灵石": "山西省", "壶关": "山西省", "雷山": "贵州省", "蒙自": "云南省",
        "勐腊": "云南省", "宁蒗": "云南省", "松潘": "四川省", "小金": "四川省", "开阳": "贵州省",
        "青阳": "安徽省", "织金": "贵州省", "中牟": "河南省", "仙居": "浙江省", "金秀": "广西壮族自治区",
        "桐乡": "浙江省","阳朔": "广西壮族自治区","大新": "广西壮族自治区","荔波": "贵州省","永修": "江西省",
        "隰县": "山西省","龙胜": "广西壮族自治区","达拉特旗": "内蒙古自治区","宜川": "陕西省","资溪": "江西省",
        "灵川": "广西壮族自治区","腾冲": "云南省","东阳": "浙江省","永登": "甘肃省","屏南": "广西壮族自治区",
        "乐清": "浙江省","兴义": "贵州省","新源": "新疆维吾尔自治区","曲阜": "山东省","门源": "青海省",
        "共和": "青海省","当雄": "西藏自治区","靖西": "广西壮族自治区","荔浦": "广西壮族自治区","轮台": "新疆维吾尔自治区",
        "平塘": "贵州省","慈利": "湖南省","桐庐": "浙江省","南靖": "福建省","敦化": "吉林省",
        "迭部": "甘肃省","永顺": "湖南省","湟源": "青海省","黎平": "贵州省","南丹": "广西壮族自治区",
        "特克斯": "新疆维吾尔自治区","文成": "浙江省","新密": "河南省","荣成": "山东省","宣汉": "四川省",
        "鹤峰": "湖北省","永济": "山西省","工布江达": "西藏自治区","宁安": "黑龙江省","建德": "浙江省",
        "新巴尔虎左旗": "内蒙古自治区","漠河": "黑龙江省","米林": "西藏自治区","德清": "浙江省","临漳": "河北省",
        "新平": "云南省","霞浦": "福建省","关岭": "贵州省","会泽": "云南省","开阳": "贵州省",
        "汤阴": "河南省","溧阳": "江苏省","易县": "河北省","泾阳": "陕西省","新昌": "浙江省",
        "从江": "贵州省","犍为": "四川省","兰溪": "浙江省","平武": "四川省","乡宁": "山西省",
        "天台": "浙江省","英德": "广东省","沂水": "山东省","鄯善": "新疆维吾尔自治区","阿克赛": "甘肃省",
        "云和": "浙江省","满洲里": "内蒙古自治区","南澳": "广东省","平潭": "福建省","黎城": "山西省",
        "福鼎": "福建省","铅山": "江西省","察右中旗": "内蒙古自治区","贵溪": "江西省","乌恰": "新疆维吾尔自治区",
        "昌黎": "河北省","巩留": "新疆维吾尔自治区","贺兰": "宁夏回族自治区","休宁": "安徽省","武安": "河北省",
        "洪洞": "山西省","平遥": "山西省","辉县": "内蒙古自治区","汶川": "四川省","丹巴": "四川省",
        "浪卡子": "西藏自治区","潜山": "安徽省","靖边": "陕西省","宜章": "湖南省","安远": "江西省",
        "昭平": "广西壮族自治区","三江": "广西壮族自治区","定日": "西藏自治区","灵武": "宁夏回族自治区","珲春": "吉林省",
        "丘北": "云南省","盘州": "贵州省","松桃": "甘肃省","歙县": "安徽省","都匀": "贵州省",
        "大方": "贵州省","若尔盖": "四川省","正定": "河北省","元阳": "云南省","阳曲": "山西省",
        "连州": "广东省","陈巴尔虎旗": "内蒙古自治区","温岭": "浙江省","嘉善": "浙江省","蒙阴": "山东省",
        "林州": "河南省","婺源": "江西省","丰宁": "河北省","凭祥": "广西壮族自治区","荥经": "四川省",
        "石台": "安徽省","镇远": "贵州省","宕昌": "甘肃省","黄陵": "陕西省","平阳": "浙江省",
        "金寨": "安徽省","长阳": "湖北省","温宿": "新疆维吾尔自治区","洪雅": "四川省","沧源": "云南省",
        "普格": "四川省","南江": "四川省","杭锦旗": "内蒙古自治区","甘泉": "陕西省","博罗": "广东省",
        "肥西": "安徽省","卓资": "内蒙古自治区","乾县": "陕西省","阳城": "河南省","新宁": "广东省",
        "长宁": "四川省","扶风": "陕西省","道孚": "四川省","额济纳旗": "内蒙古自治区","二连浩特": "内蒙古自治区",
        "清水河": "内蒙古自治区","翁牛特旗": "内蒙古自治区","奇台": "新疆维吾尔自治区","宜良": "云南省","伊宁": "新疆维吾尔自治区",
        "青铜峡": "宁夏回族自治区","涉县": "河北省","平罗": "宁夏回族自治区","伊金霍洛旗": "内蒙古自治区","新北": "台湾省",
        "惠安": "福建省","哈巴河": "新疆维吾尔自治区","宣恩": "湖北省","花垣": "湖南省","连南": "广东省",
        "和静": "新疆维吾尔自治区","泸定": "四川省","仁化": "广东省","上思": "广西壮族自治区","奎屯": "新疆维吾尔自治区",
        "闽侯": "福建省","剑川": "云南省","宾川": "云南省","合浦": "广西壮族自治区","东乌旗": "内蒙古自治区",
        "武川": "内蒙古自治区","永嘉": "浙江省","长兴": "浙江省","宁强": "陕西省","阿瓦提": "新疆维吾尔自治区",
        "赫章": "贵州省","鹿寨": "广西壮族自治区","巴塘": "四川","塔什库尔干": "新疆维吾尔自治区","永吉": "吉林省",
        "宁明": "广西壮族自治区","八宿": "西藏自治区","凌云": "广西壮族自治区","罗平": "云南省","赤壁": "湖北省",
        "茫崖": "青海省","墨江": "云南省","图们": "吉林省","滦平": "河北省","宽甸": "辽宁省",
        "弋阳": "江西省","吉首": "湖南省","天祝": "甘肃省","龙游": "浙江省","乐业": "广西壮族自治区",
        "新安": "河南省","嘉荫": "黑龙江省","平南": "广西壮族自治区","德钦": "云南省","鄱阳": "江西省",
        "开平": "广东省","介休": "山西省","根河": "内蒙古自治区","班戈": "西藏自治区","阿拉善左旗": "内蒙古自治区",
        "周至": "陕西省","平顺": "山西省","普兰": "西藏自治区","宜都": "湖北省","景泰": "甘肃省",

        # 根据景点数据添加更多城市
        "九寨沟": "四川省", "张家界": "湖南省", "峨眉山": "四川省", "黄山": "安徽省", 
        "庐山": "江西省", "华山": "陕西省", "泰山": "山东省", "衡山": "湖南省", 
        "恒山": "山西省", "嵩山": "河南省", "武当山": "湖北省", "龙虎山": "江西省", 
        "青城山": "四川省", "三清山": "江西省", "雁荡山": "浙江省", "普陀山": "浙江省", 
        "五台山": "山西省", "九华山": "安徽省", "崂山": "山东省", "武夷山": "福建省", 
        "天山": "新疆维吾尔自治区", "长白山": "吉林省", "玉龙雪山": "云南省", "梅里雪山": "云南省", 
        "贡嘎山": "四川省", "四姑娘山": "四川省", "梵净山": "贵州省", "武陵源": "湖南省", 
        "神农架": "湖北省", "大兴安岭": "黑龙江省", "小兴安岭": "黑龙江省", "秦岭": "陕西省", 
        "大巴山": "重庆市", "大别山": "安徽省", "罗霄山": "江西省", "南岭": "湖南省", 
        "乌蒙山": "贵州省", "哀牢山": "云南省", "横断山": "四川省", "祁连山": "甘肃省", 
        "昆仑山": "新疆维吾尔自治区", "唐古拉山": "青海省", "喜马拉雅山": "西藏自治区", 
        "阿里山": "台湾省", "玉山": "台湾省", "阳明山": "台湾省", "太鲁阁": "台湾省",
        
        # 根据景点名称中的地区添加
        "天安门": "北京市", "王府井": "北京市", "八达岭": "北京市", "野生动物园": "北京市",
        "外滩": "上海市", "兵马俑": "陕西省", "华清宫": "陕西省", "迪士尼": "上海市",
        "环球度假区": "北京市", "天门山": "湖南省", "熊猫基地": "四川省", "蜈支洲岛": "海南省",
        "都江堰": "四川省", "亚特兰蒂斯": "海南省", "长隆": "广东省", "涠洲岛": "广西壮族自治区",
        "赛里木湖": "新疆维吾尔自治区", "冰雪大世界": "黑龙江省", "拙政园": "江苏省",
        "恭王府": "北京市", "海洋公园": "香港特别行政区", "小七孔": "贵州省", "武侯祠": "四川省",
        "华清宫": "陕西省", "长恨歌": "陕西省", "山顶缆车": "香港特别行政区", "滕王阁": "江西省",
        "七彩丹霞": "甘肃省", "鼓浪屿": "福建省", "热带植物园": "云南省", "千户苗寨": "贵州省",
        "曼听御花园": "云南省", "湖南省博物馆": "湖南省", "橘子洲": "湖南省", "牛首山": "江苏省",
        "南山文化旅游区": "海南省", "东方明珠": "上海市", "乐山大佛": "四川省", "茶卡盐湖": "青海省",
        "分界洲岛": "海南省", "晋祠": "山西省", "黄河壶口瀑布": "山西省", "清明上河园": "河南省",
        "青城山": "四川省", "东北虎林园": "黑龙江省", "趵突泉": "山东省", "青秀山": "广西壮族自治区",
        "啤酒博物馆": "山东省", "天坛": "北京市", "滇池": "云南省", "瘦西湖": "江苏省",
        "植物园": "福建省", "宏村": "安徽省", "三清山": "江西省", "老君山": "河南省",
        "沈阳故宫": "辽宁省", "张学良旧居": "辽宁省", "失落的空间水族馆": "海南省", "平遥古城": "山西省",
        "东风韵": "云南省", "宋城": "浙江省", "普陀山": "浙江省", "两江四湖": "广西壮族自治区",
        "象鼻山": "广西壮族自治区", "颐和园": "北京市", "黄龙": "四川省", "天山天池": "新疆维吾尔自治区",
        "太平山顶": "香港特别行政区", "野象谷": "云南省", "杜甫江阁": "湖南省", "灵隐飞来峰": "浙江省",
        "钟楼": "陕西省", "四姑娘山": "四川省", "应县木塔": "山西省", "国家博物馆": "北京市",
        "武隆天生三桥": "重庆市", "武隆喀斯特": "重庆市", "虎跳峡": "云南省", "广州塔": "广东省",
        "钟鼓索道": "福建省", "布达拉宫": "西藏自治区", "喀纳斯": "新疆维吾尔自治区", "悬空寺": "山西省",
        "方城": "辽宁省", "长江索道": "重庆市", "苍山": "云南省", "九华山": "安徽省",
        "织金洞": "贵州省", "故宫文化博物馆": "香港特别行政区", "民族村": "云南省", "重庆动物园": "重庆市",
        "神仙居": "浙江省", "大唐芙蓉园": "陕西省", "陶阳里": "江西省", "慕田峪长城": "北京市",
        "独秀峰王城": "广西壮族自治区", "泸沽湖": "云南省", "婺女洲": "江西省", "稻城亚丁": "四川省",
        "崂山": "山东省", "海昌海洋公园": "上海市", "毕棚沟": "四川省", "天涯海角": "海南省",
        "藏文化博物院": "青海省", "乌镇": "浙江省", "万岁山武侠城": "河南省", "宋都皇城": "河南省",
        "大召寺": "内蒙古自治区", "逛博物馆": "陕西省", "红山森林动物园": "江苏省", "塔尔寺": "青海省",
        "海上观音": "海南省", "普达措": "云南省", "刘公岛": "山东省", "小西天": "山西省",
        "响沙湾": "内蒙古自治区", "龙脊梯田": "广西壮族自治区", "沙坡头": "宁夏回族自治区",
        "原始森林公园": "云南省", "黄河壶口瀑布": "陕西省", "熊猫谷": "四川省", "豫园": "上海市",
        "金沙遗址": "四川省", "西湖游船": "浙江省", "湖北省博物馆": "湖北省", "世界之窗": "广东省",
        "莫高窟": "甘肃省", "寒山寺": "江苏省", "平潭岛": "福建省", "上海博物馆": "上海市",
        "太湖鼋头渚": "江苏省", "嘉峪关": "甘肃省", "傣族园": "云南省", "漓江三星游船": "广西壮族自治区",
        "崇圣寺三塔": "云南省", "禾木": "新疆维吾尔自治区", "青岩古镇": "贵州省", "只有河南": "河南省",
        "华严寺": "山西省", "赏银杏": "北京市", "松花江观光索道": "黑龙江省", "千古情": "海南省",
        "雨林徒步": "云南省", "文殊坊": "四川省", "川剧变脸": "四川省", "陆家嘴": "上海市",
        "恩施大峡谷": "湖北省", "陕西历史博物馆": "陕西省", "长白山": "吉林省", "野生动物园": "云南省",
        "千岛湖": "浙江省", "和顺古镇": "云南省", "极地公园": "黑龙江省", "晴川阁": "湖北省",
        "热海风景区": "云南省", "避暑山庄": "河北省", "岳阳楼": "湖南省", "枫桥": "江苏省",
        "野生动物园": "上海市", "夜郎谷": "贵州省", "中华恐龙园": "江苏省", "横店影视城": "浙江省",
        "武功山": "江西省", "蓬莱阁": "山东省", "长安十二时辰": "陕西省", "昂坪360": "香港特别行政区",
        "海底世界": "山东省", "水墨丹霞": "甘肃省", "察尔汗盐湖": "青海省", "橄榄坝": "云南省",
        "狮子林": "江苏省", "雁荡山": "浙江省", "漓江": "广西壮族自治区", "雁门关": "山西省",
        "五台山": "山西省", "七十二奇楼": "湖南省", "高椅岭": "湖南省", "海昏侯": "江西省",
        "武当山": "湖北省", "天一阁": "浙江省", "水上雅丹": "青海省", "南岳衡山": "湖南省",
        "鹿回头": "海南省", "制药六厂": "黑龙江省", "版画博物馆": "黑龙江省", "灵隐寺": "浙江省",
        "极地海洋公园": "山东省", "洱海": "云南省", "遇龙河": "广西壮族自治区", "竹筏漂游": "广西壮族自治区",
        "翡翠湖": "青海省", "朝阳公园": "北京市", "泡泡玛特": "北京市", "野生动物园": "北京市",
        "三峡大坝": "湖北省", "净月潭": "吉林省", "乔家大院": "山西省", "雪乡": "黑龙江省",
        "王家大院": "山西省", "千古情": "广西壮族自治区", "太行山大峡谷": "山西省", "万峰林": "贵州省",
        "双桥沟": "四川省", "大唐不夜城": "陕西省", "虎丘": "江苏省", "方特": "广西壮族自治区",
        "阿尔山": "内蒙古自治区", "摩天轮": "香港特别行政区", "那拉提": "新疆维吾尔自治区", "西溪湿地": "浙江省",
        "时代少年团": "贵州省", "三孔": "山东省", "岳麓山": "湖南省", "老虎滩": "辽宁省",
        "大雁塔": "陕西省", "凤凰古城": "湖南省", "伏尔加庄园": "黑龙江省", "洪崖洞": "重庆市",
        "青海湖": "青海省", "二郎剑": "青海省", "两江游": "重庆市", "南浔古镇": "浙江省",
        "白帝城": "重庆市", "水舞间": "澳门特别行政区", "少林寺": "河南省", "锦屏山": "云南省",
        "天文馆": "上海市", "纳木措": "西藏自治区", "大明宫": "陕西省", "火山群": "海南省",
        "海洋水族馆": "上海市", "通灵大峡谷": "广西壮族自治区", "珠江夜游": "广东省",
        "印象刘三姐": "广西壮族自治区", "大马戏": "广东省", "长隆度假区": "广东省", "飞船乐园": "广东省",
        "留园": "江苏省", "开封府": "河南省", "看大熊猫": "四川省", "钟山": "江苏省",
        "成都博物馆": "四川省", "银子岩": "广西壮族自治区", "北京动物园": "北京市",
        "仰天湖": "湖南省", "西山": "云南省", "世界魔鬼城": "新疆维吾尔自治区", "中国天眼": "贵州省",
        "缙云仙都": "浙江省", "西部影城": "宁夏回族自治区", "西宁野生动物园": "青海省",
        "沈阳方特": "辽宁省", "张家界大峡谷": "湖南省", "维多利亚港": "香港特别行政区",
        "明仕田园": "广西壮族自治区", "碑林博物馆": "陕西省", "龟山汉墓": "江苏省",
        "两坝一峡": "湖北省", "太阳河森林公园": "云南省", "云台山": "河南省", "仙女山": "重庆市",
        "漫花庄园": "四川省", "赏红叶": "北京市", "OMG心跳乐园": "浙江省", "东湖": "湖北省",
        "云水谣": "福建省", "肇兴侗寨": "贵州省", "ZEROBASEONE": "香港特别行政区", "千古情": "云南省",
        "雍和宫": "北京市", "周庄": "江苏省", "山海关": "河北省", "日月山": "青海省",
        "航母主题公园": "天津市", "西安鼓楼": "陕西省", "扎尕那": "甘肃省", "BLACKPINK": "香港特别行政区",
        "圆明园": "北京市", "勐焕大金塔": "云南省", "湖广会馆": "重庆市", "上海中心": "上海市",
        "西递": "安徽省", "乌镇西栅": "浙江省", "松赞林": "云南省", "玉龙雪山索道": "云南省",
        "西安千古情": "陕西省", "赳赳大秦": "陕西省", "峒那屿湾": "广西壮族自治区", "陶瓷博物馆": "江西省",
        "澳门旅游塔": "澳门特别行政区", "木府": "云南省", "东江湖": "湖南省", "胡里山炮台": "福建省",
        "芙蓉镇": "湖南省", "山西博物院": "山西省", "漓江四星游船": "广西壮族自治区", "火山地质公园": "内蒙古自治区",
        "连岛": "江苏省", "如意峰": "广西壮族自治区", "喀拉峻": "新疆维吾尔自治区", "展馆深度游览": "上海市",
        "太原植物园": "山西省", "乌江寨": "贵州省", "建水小火车": "云南省", "千佛山": "山东省",
        "云和梯田": "浙江省", "三峡人家": "湖北省", "个园": "江苏省", "百丈漈": "浙江省",
        "大足石刻": "重庆市", "清源山": "福建省", "成都自然博物馆": "四川省", "新濠天地": "澳门特别行政区",
        "神雕山野生动物园": "山东省", "银滩": "广西壮族自治区", "华晨宇": "海南省", "银基动物王国": "河南省",
        "明孝陵": "江苏省", "满洲里国门": "内蒙古自治区", "西夏陵": "宁夏回族自治区", "洛阳博物馆": "河南省",
        "朱家尖": "浙江省", "观音法界": "浙江省", "平山湖大峡谷": "甘肃省", "大理古城": "云南省",
        "丽江古城": "云南省", "呀诺达": "海南省", "南澳岛": "广东省", "长隆欢乐世界": "广东省",
        "镜泊湖": "黑龙江省", "黄浦江游览": "上海市", "驼铃传奇": "陕西省", "应天门": "河南省",
        "云端之眼": "重庆市", "巴松措": "西藏自治区", "上海千古情": "上海市", "勐泐文化旅游区": "云南省",
        "乐高乐园": "上海市", "汪苏泷": "浙江省", "蜀风雅韵": "四川省", "长白山北坡": "吉林省",
        "西安博物院": "陕西省", "古窑民俗博览区": "江西省", "乌兰布统": "内蒙古自治区", "何园": "江苏省",
        "天河潭": "贵州省", "游艇俱乐部": "海南省", "呈坎": "安徽省", "中俄边境": "内蒙古自治区",
        "摩天轮": "澳门特别行政区", "仙人井": "福建省", "剑门关": "四川省", "河南博物院": "河南省",
        "扎龙": "黑龙江省", "王力宏": "江苏省", "福建土楼": "福建省", "兴隆热带植物园": "海南省",
        "琅琊古城": "山东省", "永定土楼": "福建省", "磁浮列车": "上海市", "鹳雀楼": "山西省",
        "长江": "湖北省", "厦门科技馆": "福建省", "重庆欢乐谷": "重庆市", "漓江竹筏": "广西壮族自治区",
        "屏山峡谷": "湖北省", "岳麓书院": "湖南省", "白马寺": "河南省", "三坊七巷": "福建省",
        "森林动物园": "辽宁省", "景山公园": "北京市", "武陵源": "湖南省", "68海里": "福建省",
        "云南省博物馆": "云南省", "上海之巅": "上海市", "北极村": "黑龙江省", "呼伦贝尔": "内蒙古自治区",
        "周深": "广东省", "新疆博物馆": "新疆维吾尔自治区", "雅鲁藏布大峡谷": "西藏自治区",
        "亚特兰蒂斯C秀": "海南省", "大明湖": "山东省", "天下第一泉": "山东省", "夫子庙": "江苏省",
        "台北故宫": "台湾省", "莫干山": "浙江省", "梭布垭石林": "湖北省", "古龙山大峡谷": "广西壮族自治区",
        "天安门广场": "北京市", "恒山": "山西省", "中华大熊猫苑": "四川省", "龙虎山": "江西省",
        "大报恩寺": "江苏省", "厦门大学": "福建省", "辉腾锡勒": "内蒙古自治区", "黄花沟": "内蒙古自治区",
        "火山地热": "云南省", "昆明老街": "云南省", "马岭河峡谷": "贵州省", "芦笛岩": "广西壮族自治区",
        "葛仙村": "江西省", "平安金融中心": "广东省", "黄龙洞": "湖南省", "夫子庙秦淮风光带": "江苏省",
        "观光巴士": "香港特别行政区", "世界之窗": "湖南省", "拈花湾": "江苏省", "太姥山": "福建省",
        "新濠影汇": "澳门特别行政区", "天山神秘大峡谷": "新疆维吾尔自治区", "科学技术馆": "北京市",
        "奥林匹克公园": "北京市", "南越王博物院": "广东省", "恩施土司城": "湖北省", "太阳岛": "黑龙江省",
        "东山岛": "福建省", "开封博物馆": "河南省", "开元森泊": "山东省", "北海湿地": "云南省",
        "明堂天堂": "河南省", "隋唐洛阳城": "河南省", "阿那亚": "河北省", "M+": "香港特别行政区",
        "花果山": "江苏省", "库尔德宁": "新疆维吾尔自治区", "两江游览": "湖北省", "印象丽江": "云南省",
        "上海动物园": "上海市", "猴研岛": "福建省", "望天树": "云南省", "苏州博物馆": "江苏省",
        "贺兰山岩画": "宁夏回族自治区", "昆明旅游索道": "云南省", "天心阁": "湖南省", "齐云山": "安徽省",
        "北海海底世界": "广西壮族自治区", "南山寺": "海南省", "卧龙大熊猫": "四川省", "大理骑行": "云南省",
        "成都动物园": "四川省", "湄洲岛": "福建省", "南湾猴岛": "海南省", "龙宫": "贵州省",
        "印象大红袍": "福建省", "桂海晴岚": "广西壮族自治区", "鹅泉": "广西壮族自治区", "陈家祠": "广东省",
        "莽山五指峰": "湖南省", "电影公社": "海南省", "海洋欢乐世界": "海南省", "Hello Kitty乐园": "浙江省",
        "蜀南竹海": "四川省", "成都欢乐谷": "四川省", "波浪谷": "陕西省", "天柱山": "安徽省",
        "三峡晓峰": "湖北省", "三峡大瀑布": "湖北省", "三门海": "广西壮族自治区", "魅力湘西": "湖南省",
        "巫山小三峡": "重庆市", "大明寺": "江苏省", "816工程": "重庆市", "南宁动物园": "广西壮族自治区",
        "一寨两国": "云南省", "明十三陵": "北京市", "回民街": "陕西省", "高家大院": "陕西省",
        "蓝月谷": "云南省", "圣亚海洋世界": "辽宁省", "双林寺": "山西省", "宝泉": "河南省",
        "九寨千古情": "四川省", "魅力湘西": "湖南省", "甘肃省博物馆": "甘肃省", "夜游锦江": "四川省",
        "广胜寺": "山西省", "大佛寺": "甘肃省", "九曲溪竹筏": "福建省", "又见平遥": "山西省",
        "大同古城": "山西省", "亚布力": "黑龙江省", "阳光度假村": "黑龙江省", "三峡之巅": "重庆市",
        "湘江游轮": "湖南省", "总督楼": "山东省", "港珠澳大桥": "广东省", "古堰画乡": "浙江省",
        "秦岭野生动物园": "陕西省", "羊卓雍错": "西藏自治区", "同里古镇": "江苏省", "考古博物馆": "陕西省",
        "华夏文旅": "陕西省", "天津之眼": "天津市", "老龙头": "河北省", "鹭江夜游": "福建省",
        "厦门海上游": "福建省", "火焰山": "新疆维吾尔自治区", "江西省博物馆": "江西省", "南京博物院": "江苏省",
        "南昌之星": "江西省", "八大关": "山东省", "又见敦煌": "甘肃省", "拉市海": "云南省",
        "西游乐园": "江苏省", "巴拉格宗": "云南省", "黄姚古镇": "广西壮族自治区", "重庆1949": "重庆市",
        "鸽子窝": "河北省", "尼山": "山东省", "尼山圣境": "山东省", "水上乐园": "澳门特别行政区",
        "程阳八寨": "广西壮族自治区", "独山子大峡谷": "新疆维吾尔自治区", "槟榔谷": "海南省",
        "徐州博物馆": "江苏省", "开元寺": "福建省", "雷峰塔": "浙江省", "珠峰大本营": "西藏自治区",
        "山东博物馆": "山东省", "水洞沟": "宁夏回族自治区", "徽州古城": "安徽省", "迪士尼小镇": "上海市",
        "什刹海": "北京市", "鲁迅故里": "浙江省", "绍兴古城": "浙江省", "古滇度假区": "云南省",
        "欢乐世界": "云南省", "铜仁大峡谷": "贵州省", "海底世界": "湖南省", "马蹄石窟": "甘肃省",
        "国家海洋博物馆": "天津市", "忻州古城": "山西省", "铜鼓岭": "海南省", "木格措": "四川省",
        "台儿庄古城": "山东省", "解放碑": "重庆市", "普者黑": "云南省", "茶博园": "福建省",
        "鸟巢": "北京市", "垂云通天河": "浙江省", "文殊院": "四川省", "辽宁省博物馆": "辽宁省",
        "乌蒙大草原": "贵州省", "十五运会": "澳门特别行政区", "长江三峡": "湖北省", "双廊": "云南省",
        "MAMA": "香港特别行政区", "大慈恩寺": "陕西省", "总统府": "江苏省", "香妃园": "新疆维吾尔自治区",
        "热带野生动植物园": "海南省", "七星景区": "广西壮族自治区", "河头老街": "河北省", "广济桥": "广东省",
        "腾龙洞": "湖北省", "宽窄巷子": "四川省", "北京欢乐谷": "北京市", "参观四合院": "北京市",
        "百里杜鹃": "贵州省", "银基国际旅游度假区": "河南省", "地心谷": "湖北省", "黄河九曲第一湾": "四川省",
        "四川博物院": "四川省", "云南映象": "云南省", "阳关": "甘肃省", "赏银杏": "四川省",
        "伪满皇宫": "吉林省", "王力宏": "四川省", "隆兴寺": "河北省", "嵩山": "河南省",
        "席力图召": "内蒙古自治区", "元阳哈尼梯田": "云南省", "长影世纪城": "吉林省", "看日出": "福建省",
        "感通索道": "云南省", "沈园": "浙江省", "citywalk": "北京市", "海上云台山": "江苏省",
        "仙螺岛": "河北省", "南戴河": "河北省", "珍奇园": "云南省", "相公山": "广西壮族自治区",
        "长隆秀": "广东省", "江郎山": "浙江省", "龙头山": "陕西省", "朱家花园": "云南省",
        "西岭雪山": "四川省", "百龙天梯": "湖南省", "瓷房子": "天津市", "李自健美术馆": "湖南省",
        "锦绣中华": "广东省", "川剧变脸": "四川省", "汪苏泷": "广西壮族自治区", "成山头": "山东省",
        "西霞口": "山东省", "国际大巴扎": "新疆维吾尔自治区", "地质博物馆": "北京市", "法门寺": "陕西省",
        "广西博物馆": "广西壮族自治区", "太原古县城": "山西省", "海南省博物馆": "海南省", "洱海游船": "云南省",
        "太平山索道": "山东省", "夏塔": "新疆维吾尔自治区", "上海欢乐谷": "上海市", "连州地下河": "广东省",
        "海钓俱乐部": "海南省", "太行大峡谷": "河南省", "柯岩": "浙江省", "阆中古城": "四川省",
        "可可托海": "新疆维吾尔自治区", "墨石公园": "四川省", "惠山古镇": "江苏省", "龙水峡地缝": "重庆市",
        "六朝博物馆": "江苏省", "莫尔格勒河": "内蒙古自治区", "铜官窑": "湖南省", "青州古城": "山东省",
        "长屿硐天": "浙江省", "西塘古镇": "浙江省", "普洱茶马古道": "云南省", "黄河楼": "甘肃省",
        "代王府": "山西省", "若尔盖花湖": "四川省", "电影小镇": "湖南省", "莲宝叶则": "四川省",
        "友谊关": "广西壮族自治区", "百里天路": "河北省", "绵阳方特": "四川省", "婺源风景区": "江西省",
        "云上草原": "浙江省", "寻迹洛神赋": "河南省", "亚龙湾": "海南省", "大河荟": "河南省",
        "古隆中": "湖北省", "闽江夜游": "福建省", "游艇俱乐部": "海南省", "束河古镇": "云南省",
        "百魔洞": "广西壮族自治区", "牯牛降": "安徽省", "动植物公园": "吉林省", "巴厘村": "海南省",
        "石梅湾": "海南省", "中环": "香港特别行政区", "黑神话": "山西省", "木偶戏": "福建省",
        "星海公园": "辽宁省", "南少林寺": "福建省", "东湖": "浙江省", "官鹅沟": "甘肃省",
        "镇远古城": "贵州省", "告庄西双景": "云南省", "科学中心": "广东省", "黄帝陵": "陕西省",
        "城隍庙": "上海市", "磁器口": "重庆市", "喜洲": "云南省", "殷墟": "河南省",
        "天堂寨": "安徽省", "邓紫棋": "广西壮族自治区", "清江画廊": "湖北省", "变脸秀": "四川省",
        "川剧院": "四川省", "托木尔": "新疆维吾尔自治区", "信号山": "山东省", "金海湾红树林": "广西壮族自治区",
        "大相国寺": "河南省", "八仙过海": "山东省", "清河半岛": "辽宁省", "圣索菲亚教堂": "黑龙江省",
        "索菲亚广场": "黑龙江省", "南普陀寺": "福建省", "小东江": "湖南省", "金山鹿苑": "黑龙江省",
        "秦淮河画舫": "江苏省", "喀什古城": "新疆维吾尔自治区", "野生动物园": "云南省", "民族风情园": "云南省",
        "净土寺": "山西省", "云上海天": "山东省", "洗马潭索道": "云南省", "涠洲岛游船": "广西壮族自治区",
        "世外桃源": "广西壮族自治区", "日光岩": "福建省", "春熙路": "四川省", "长岛": "山东省",
        "田螺坑": "福建省", "善化寺": "山西省", "甘泉大峡谷": "陕西省", "波塞冬": "黑龙江省",
        "海底世界": "黑龙江省", "太阳宫": "云南省", "山塘街": "江苏省", "皇城相府": "山西省",
        "尖沙咀": "香港特别行政区", "九龙": "香港特别行政区", "提线木偶": "福建省", "玉门关": "甘肃省",
        "热带花卉园": "云南省", "广府古城": "河北省", "鄂尔多斯草原": "内蒙古自治区", "黔灵山": "贵州省",
        "国花园": "河南省", "星光夜市": "云南省", "兴汉胜境": "陕西省", "夜上黄鹤楼": "湖北省",
        "水果之窗": "海南省", "葡萄沟": "新疆维吾尔自治区", "库车王府": "新疆维吾尔自治区", "teamLab": "澳门特别行政区",
        "抚仙湖": "云南省", "乾陵": "陕西省", "海花岛": "海南省", "十里画廊": "广西壮族自治区",
        "兰亭": "浙江省", "雅丹地质公园": "甘肃省", "张杰": "浙江省", "龙泉古镇": "河北省",
        "奥帆中心": "山东省", "海滨风景区": "山东省", "古北水镇": "北京市", "笔架山": "辽宁省",
        "广东省博物馆": "广东省", "周深": "香港特别行政区", "贺兰山森林公园": "宁夏回族自治区",
        "丰都鬼城": "重庆市", "天星小轮": "香港特别行政区", "天山大峡谷": "新疆维吾尔自治区",
        "人民大礼堂": "重庆市", "温泉瀑布": "四川省", "三娘湾": "广西壮族自治区", "北海公园": "北京市",
        "珠峰": "西藏自治区", "永泉小镇": "安徽省", "双龙": "浙江省", "深圳欢乐谷": "广东省"
    }
    
    return city_province_map

def clean_city_name(city_name):
    """
    清洗城市名称，去除常见后缀和修饰词
    """
    if pd.isna(city_name) or city_name == "":
        return ""
    
    city_str = str(city_name).strip()
    
    # 去除常见后缀
    suffixes = ['市', '县', '区', '镇', '乡', '村', '景区', '景点', '旅游区', '风景区', '度假区', 
                '公园', '博物馆', '遗址', '古城', '古镇', '山庄', '寺庙', '寺', '庙', '庵',
                '峡谷', '山脉', '山', '湖', '河', '海', '岛', '湾', '滩', '林', '园', '苑']
    
    for suffix in suffixes:
        if city_str.endswith(suffix):
            city_str = city_str[:-len(suffix)]
    
    return city_str.strip()

def process_city_data(input_file, output_file):
    """
    处理城市数据，添加省份信息
    """
    try:
        # 检测文件编码
        print("正在检测文件编码...")
        file_encoding = detect_encoding(input_file)
        
        # 尝试用检测到的编码读取文件
        try:
            df = pd.read_csv(input_file, encoding=file_encoding)
            print(f"使用 {file_encoding} 编码成功读取文件")
        except UnicodeDecodeError:
            # 如果检测的编码失败，尝试其他常见编码
            encodings_to_try = ['gbk', 'gb2312', 'gb18030', 'utf-8', 'latin1']
            for encoding in encodings_to_try:
                try:
                    df = pd.read_csv(input_file, encoding=encoding)
                    print(f"使用 {encoding} 编码成功读取文件")
                    file_encoding = encoding
                    break
                except UnicodeDecodeError:
                    continue
            else:
                raise Exception("无法找到正确的文件编码")
        
        print(f"成功读取文件，共{len(df)}条数据")
        
        # 获取城市列
        if '城市' not in df.columns:
            print("错误：未找到'城市'列")
            print(f"文件中的列名: {df.columns.tolist()}")
            return
        
        # 获取城市-省份映射
        print("正在创建城市-省份映射...")
        city_province_map = get_city_province_mapping()
        print(f"成功创建{len(city_province_map)}条城市-省份映射")
        
        # 添加省份信息
        provinces = []
        unmatched_cities = set()
        
        for city in df['城市']:
            if pd.isna(city) or city == "":
                provinces.append("未知")
                continue
            
            city_str = str(city).strip()
            cleaned_city = clean_city_name(city_str)
            
            # 尝试多种匹配方式
            matched = False
            
            # 1. 直接匹配原始城市名称
            if city_str in city_province_map:
                provinces.append(city_province_map[city_str])
                matched = True
            # 2. 匹配清洗后的城市名称
            elif cleaned_city and cleaned_city in city_province_map:
                provinces.append(city_province_map[cleaned_city])
                matched = True
            # 3. 模糊匹配
            else:
                for map_city, province in city_province_map.items():
                    if (map_city in city_str or city_str in map_city or
                        (cleaned_city and (map_city in cleaned_city or cleaned_city in map_city))):
                        provinces.append(province)
                        matched = True
                        break
            
            if not matched:
                provinces.append("未知")
                unmatched_cities.add(city_str)
        
        # 添加新列
        df['省份'] = provinces
        
        # 保存结果
        df.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"处理完成！结果已保存到: {output_file}")
        
        # 显示统计信息
        province_counts = df['省份'].value_counts()
        print("\n省份分布统计:")
        for province, count in province_counts.items():
            print(f"{province}: {count}个景点")
        
        # 显示未匹配的城市
        if unmatched_cities:
            print(f"\n未匹配的城市 ({len(unmatched_cities)}个):")
            for city in sorted(unmatched_cities):
                print(f"  {city}")
        else:
            print("\n所有城市都已成功匹配!")
            
    except Exception as e:
        print(f"处理过程中出现错误: {e}")
        import traceback
        traceback.print_exc()

# 主程序
if __name__ == "__main__":
    # 设置文件路径
    input_file = r"D:\2025 at CityU\semester A\5507 Soc Meida data\final\5507数据\3000景点清洗后数据.csv"
    output_dir = r"D:\2025 at CityU\semester A\5507 Soc Meida data\final\5507数据"
    output_file = os.path.join(output_dir, "3000景点带省份信息.csv")
    
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"输入文件: {input_file}")
    print(f"输出目录: {output_dir}")
    print(f"输出文件: {output_file}")
    print("-" * 50)
    
    process_city_data(input_file, output_file)

输入文件: D:\2025 at CityU\semester A\5507 Soc Meida data\final\5507数据\3000景点清洗后数据.csv
输出目录: D:\2025 at CityU\semester A\5507 Soc Meida data\final\5507数据
输出文件: D:\2025 at CityU\semester A\5507 Soc Meida data\final\5507数据\3000景点带省份信息.csv
--------------------------------------------------
正在检测文件编码...
检测到文件编码: GB2312 (置信度: 0.99)
使用 gbk 编码成功读取文件
成功读取文件，共3002条数据
正在创建城市-省份映射...
成功创建1582条城市-省份映射
处理完成！结果已保存到: D:\2025 at CityU\semester A\5507 Soc Meida data\final\5507数据\3000景点带省份信息.csv

省份分布统计:
云南省: 228个景点
浙江省: 178个景点
四川省: 170个景点
广东省: 158个景点
江苏省: 158个景点
山东省: 135个景点
广西壮族自治区: 120个景点
福建省: 119个景点
海南省: 115个景点
新疆维吾尔自治区: 98个景点
陕西省: 96个景点
湖北省: 94个景点
湖南省: 94个景点
贵州省: 89个景点
北京市: 89个景点
内蒙古自治区: 88个景点
山西省: 85个景点
河南省: 83个景点
江西省: 79个景点
上海市: 75个景点
黑龙江省: 71个景点
河北省: 67个景点
重庆市: 67个景点
甘肃省: 66个景点
辽宁省: 63个景点
香港特别行政区: 58个景点
安徽省: 57个景点
吉林省: 48个景点
澳门特别行政区: 37个景点
西藏自治区: 29个景点
青海省: 26个景点
天津市: 20个景点
台湾省: 18个景点
宁夏回族自治区: 17个景点
未知: 6个景点
四川: 1个景点

未匹配的城市 (6个):
  3000
  兴城
  松阳
  漳浦
  罗城
  阿克塞
